In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
from IPython.display import Image
from keras.preprocessing import image
from keras import optimizers
from keras import layers,models
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
print(os.listdir("../input"))

import numpy as np


In [ ]:
train_dir="../input/train/train"
test_dir="../input/test/test"
train=pd.read_csv('../input/train.csv')

df_test=pd.read_csv('../input/sample_submission.csv')


In [ ]:
train.head(5)


In [ ]:
print('out dataset has {} rows and {} columns'.format(train.shape[0],train.shape[1]))


In [ ]:
train['has_cactus'].value_counts()


In [ ]:
print("The number of rows in test set is %d"%(len(os.listdir('../input/test/test'))))


In [ ]:
Image(os.path.join("../input/train/train",train.iloc[0,0]),width=250,height=250)


In [ ]:
def prepare_data(data,m,direc):
    print('preparing data')
    X_train=np.zeros((m,150,150,3))
    count=0
    for fig in data['id']:
        img=image.load_img(os.path.join(direc,fig),target_size=(150,150,3))
        x=image.img_to_array(img)
        x=preprocess_input(x)
        X_train[count]=x/255
        count+=1
        
        
    print("Done")
    return X_train

        
        
    


In [ ]:
data=prepare_data(train,train.shape[0],train_dir)
test=prepare_data(df_test,len(df_test),test_dir)


In [ ]:
X_train=data[:15001]
y_train=train['has_cactus'][:15001]
valid=data[15001:]
y_valid=train['has_cactus'][15001:]


In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
         


In [ ]:
model.summary()


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.rmsprop(),metrics=['acc'])


In [ ]:
epochs=10
history=model.fit(X_train,y_train,epochs=10,validation_data=(valid,y_valid))


In [ ]:
acc=history.history['acc']
epochs_=range(0,epochs)
plt.plot(epochs_,acc,label='training accuracy')

acc_val=history.history['val_acc']
plt.scatter(epochs_,acc_val,label="validation accuracy")

plt.legend()




    


In [ ]:
acc=history.history['loss']
epochs_=range(0,epochs)
plt.plot(epochs_,acc,label='training loss')

acc_val=history.history['val_loss']
plt.scatter(epochs_,acc_val,label="validation loss")

plt.legend()


In [ ]:
y_pre=model.predict_proba(test)


In [ ]:
df=pd.DataFrame({'id':df_test['id'] })
df['has_cactus']=y_pre
df.to_csv("submission.csv",index=False)
